In [2]:
import array
import awkward as ak
import mplhep
import glob
import random
import os
import order as od
import matplotlib as mpl
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pickle
import hist
import json

# CMS style for plots
mplhep.style.use(mplhep.style.CMS)

# enable JavaScript in ROOT plots
%matplotlib inline

  Using cached order-2.1.4-py3-none-any.whl


In [13]:
#
# ANALYSIS AND DATASET
#

analysis = od.Analysis(name="monotop", id=1)

for i, era in enumerate(["2016preVFP", "2016postVFP", "2017", "2018"]):
    cpn = od.Campaign(
        name=era,
        id=10 * i,
    )

    cpn.add_dataset(
        name="z_2_jets_nu_nu_pt_250_to_400_amcatnlo",
        id=1000 * i + 100,
        label="Z #to #nu#nu + 2 Jets, p_{T,Z} #in [250, 400] GeV",
        keys=[],
        aux=dict(
            fileglob="root://dcache-cms-xrootd.desy.de:1094/pnfs/desy.de/cms/tier2/store/user/mwassmer/ntuples_MonoTop_Aug23/{}/Z2JetsToNuNu_M-50_LHEFilterPtZ-250To400_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/tree_*.root".format(era),
            columns=[
                "N_AK15Jets_nom",
                "N_Jets_nom",
                "N_LooseMuons",
                "N_LooseElectrons",
                "N_LoosePhotons",
                "Hadr_Recoil_MET_T1XY_Pt_nom",
                "AK15Jet_Pt_nom",
                "Weight_GEN",
                "Weight_muRDown",
                "Weight_muRUp",
                "Weight_muFDown",
                "Weight_muFUp",                
            ]
        )
    )
    
    analysis.add_config(name="monotop_{}".format(era), campaign=cpn)

In [14]:
#
# VARIABLES
#

h_t = od.Variable(
    name="Hadr_Recoil_MET_T1XY_Pt_nom",
    x_title="Hadronic Recoil H_{{T}}",
    x_title_short="H_{{T}}",
    unit="GeV",
    unit_format="{title} ({unit})",
    binning=[350 + 25 * i for i in range(6)] + [550, 650, 1000],
)

w_gen = od.Variable(
    name="NomWeight_Weight_GEN",
    expression="Weight_GEN",
    x_title="Generator Weight w_{{gen}}",
    x_title_short="w_{{gen}}",
    binning=(20, -100, 100),
)

w_mu_r_down_tot = od.Variable(
    name="VariationWeight_Weight_muRDown",
    expression="Weight_GEN * Weight_muRDown",
    x_title="Variation Weight w_{{gen}}w_{{#mu,R,down}}",
    x_title_short="w_{{gen}}w_{{#mu,R,down}}",
    binning=(100, -8000, 8000),
)

w_mu_r_up_tot = od.Variable(
    name="VariationWeight_Weight_muRUp",
    expression="Weight_GEN * Weight_muRUp",
    x_title="Variation Weight w_{{gen}}w_{{#mu,R,up}}",
    x_title_short="w_{{gen}}w_{{#mu,R,up}}",
    binning=(100, -8000, 8000),
)

w_mu_f_down_tot = od.Variable(
    name="VariationWeight_Weight_muFDown",
    expression="Weight_GEN * Weight_muFDown",
    x_title="Variation Weight w_{{gen}}w_{{#mu,F,down}}",
    x_title_short="w_{{gen}}w_{{#mu,F,down}}",
    binning=(100, -8000, 8000),
)

w_mu_f_up_tot = od.Variable(
    name="VariationWeight_Weight_muFUp",
    expression="Weight_GEN * Weight_muFUp",
    x_title="Variation Weight w_{{gen}}w_{{#mu,F,down}}",
    x_title_short="w_{{gen}}w_{{#mu,F,up}}",
    binning=(100, -8000, 8000),
)

In [15]:
#
# SHIFTS
#

nominal = od.Shift(
    name="nominal",
    id=10,
    label="nominal",
    aux=dict(
        weight="Weight_GEN",
    )
)

mu_r_down = od.Shift(
    name="mu_r_down",
    id=21,
    label="$\\mu_{{\\mathrm{{R}}}}$ down",
    aux=dict(
        weights="Weight_muRDown",
    ),
)

mu_r_up = od.Shift(
    name="mu_r_up",
    id=22,
    label="$\\mu_{{\\mathrm{{R}}}}$ up",
    aux=dict(
        weight="Weight_muRUp",
    ),
)

mu_f_down = od.Shift(
    name="mu_f_down",
    id=31,
    label="$\\mu_{{\\mathrm{{F}}}}$ down",
    aux=dict(
        weight="Weight_muFDown",
    ),
)

mu_f_up = od.Shift(
    name="mu_f_up",
    id=32,
    label="$\\mu_{{\\mathrm{{F}}}}$ up",
    aux=dict(
        weight="Weight_muFUp",
    ),
)

In [20]:
def load_events(dataset, debug=False):
    fileglob = dataset.x.fileglob
    #if debug:
    #    fileglob = random.choice(glob.glob(fileglob))
    events = uproot.open(fileglob, dataset.x.columns, report=True, num_workers=6)["Events"]
    return events


def select_signal_category(events):
    mask_n_jets = (
        (events["N_AK15Jets_nom"] >= 1) &
        (events["N_Jets_nom"] >= 1)
    )
    events_sel = events[mask_n_jets]
    return events_sel[
        (events_sel["N_LooseMuons"] == 0) &
        (events_sel["N_LooseElectrons"] == 0) &
        (events_sel["N_LoosePhotons"] == 0) &
        (events_sel["Hadr_Recoil_MET_T1XY_Pt_nom"] >= 350.) &
        (events_sel["AK15Jet_Pt_nom"][:, 0] >= 250.)
    ]

In [21]:
def create_histogram(events, variable, nominal, shifts):

    # create histogram
    h = hist.Hist(
        hist.axis.StrCategory([nominal.name, ] + [shift.name for shift in shifts], name="shift", label="Shift"),
        hist.axis.Variable(variable.bin_edges, name=variable.name, label=variable.get_full_x_title()),
        storage=hist.storage.Weight(),
    )

    # fill nominal histogram
    h[{"shift": nominal.name}].fill(ak.ravel(events[variable.name]), weights=events[nominal.x.weight])

    # fill shift templates
    for shift in shifts:
        h[{"shift": shift.name}].fill(ak.ravel(events[variable.name]), weights=events[nominal.x.weight] * events[shift.x.weight])

    return h

In [22]:
events = load_events(
    analysis.get_config("monotop_{}".format(era)).campaign.get_dataset("z_2_jets_nu_nu_pt_250_to_400_amcatnlo"),
    debug=False,
)

TypeError: object_cache must be None, a MutableMapping, or an int

In [19]:
counts = []
import pprint

for array, report in events:
    
    filepath = report.file_path
    array = select_signal_category(array)

    n_tot = ak.num(array["Weight_GEN"], axis=0)
    n_mu_r_down_anomalous = ak.sum(abs(array["Weight_GEN"] * array["Weight_muRDown"]) > 2500)
    n_mu_r_up_anomalous = ak.sum(abs(array["Weight_GEN"] * array["Weight_muRUp"]) > 2500)    
    n_mu_f_down_anomalous = ak.sum(abs(array["Weight_GEN"] * array["Weight_muFDown"]) > 2500)
    n_mu_f_up_anomalous = ak.sum(abs(array["Weight_GEN"] * array["Weight_muFUp"]) > 2500)    

    #h = create_histogram(array, h_t, nominal, [mu_r_down, mu_r_up, mu_f_down, mu_f_up])

    counts.append({
        "file": filepath,
        "n_total": int(n_tot),
        "n_mu_r_down_anomalous": int(n_mu_r_down_anomalous),
        "n_mu_r_up_anomalous": int(n_mu_r_up_anomalous),
        "n_mu_f_down_anomalous": int(n_mu_f_down_anomalous),
        "n_mu_f_up_anomalous": int(n_mu_f_up_anomalous),
    })

    #with open("histograms/h_{}.pickle".format(os.path.splitext(os.path.basename(filepath))[0]), mode="w") as f:
    #    pickle.dump(h, f)

    with open("anomalous_mur_muf_weights.json", mode="w") as f:
        json.dump(counts, f)

    #h.plot()
    pprint.pprint(counts)

FileNotFoundError: file not found ([ERROR] Server responded with an error: [3011] No such file)

    'root://dcache-cms-xrootd.desy.de:1094/pnfs/desy.de/cms/tier2/store/user/mwassmer/ntuples_MonoTop_Aug23/2018/Z2JetsToNuNu_M-50_LHEFilterPtZ-250To400_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/tree_*.root'

Files may be specified as:
   * str/bytes: relative or absolute filesystem path or URL, without any colons
         other than Windows drive letter or URL schema.
         Examples: "rel/file.root", "C:\abs\file.root", "http://where/what.root"
   * str/bytes: same with an object-within-ROOT path, separated by a colon.
         Example: "rel/file.root:tdirectory/ttree"
   * pathlib.Path: always interpreted as a filesystem path or URL only (no
         object-within-ROOT path), regardless of whether there are any colons.
         Examples: Path("rel:/file.root"), Path("/abs/path:stuff.root")

Functions that accept many files (uproot.iterate, etc.) also allow:
   * glob syntax in st